In [1]:
import tensorflow as tf
import dill
import contrastive
from contrastive import utils as contrastive_utils
from acme import specs
import jax
import numpy as np

import imageio #<-- This is an internal google library, but imageio should be able to do similar things
from base64 import b64encode
from IPython.display import display, HTML
import tempfile
import tqdm
import matplotlib.pyplot as plt
import jax.numpy as jnp
import gym
from acme.wrappers import gym_wrapper

import metaworld
# from contrastive_rl import classifier_envs
from acme.wrappers import gym_wrapper
from acme.wrappers import step_limit
# from contrastive_rl import classifier_utils
# from metaworld.envs.mujoco.utils import rotation

2022-08-03 00:57:00.848920: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/sarthak/anaconda3/envs/contrastive_rl/lib/:/home/sarthak/.mujoco/mujoco200/bin
2022-08-03 00:57:00.848950: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
No module named 'flow'
No module named 'dm_control'
No module named 'carla'
pybullet build time: May 20 2022 19:45:31
/home/sarthak/anaconda3/envs/contrastive_rl/lib/python3.9/site-packages/gym/logger.py:34: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize("%s: %s" % ("WARN", msg % args), "yellow"))


In [2]:
filename = './checkpoints/learner/'  # <-- Of course, this will be different for you
reader = tf.train.load_checkpoint(filename)
params = reader.get_tensor('learner/.ATTRIBUTES/py_state')
state = dill.loads(params)

In [3]:
env, obs_dim = contrastive_utils.make_environment('fixed-goal-point_Cross', start_index=0, end_index=-1, seed=0)
spec = specs.make_environment_spec(env)
networks = contrastive.make_networks(spec, obs_dim,
                             actor_min_std=1e-6,  # <-- Make sure this is the same value used for training.
                             use_image_obs=False)

key = jax.random.PRNGKey(0)

@jax.jit
def _act(obs, key):
  dist_params = networks.policy_network.apply(
    state.policy_params, obs[None])
  key, rng = jax.random.split(key, 2)
  # action = dist_params.mode()  # <-- Use this if you want to act deterministically
  action = networks.sample(dist_params, rng)
  return action[0], key

ts = env.reset()
_act(ts.observation, key)

/home/sarthak/anaconda3/envs/contrastive_rl/lib/python3.9/site-packages/gym/logger.py:34: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize("%s: %s" % ("WARN", msg % args), "yellow"))


In here! fixed-goal-point_Cross Cross
Inside the right IF ELSE


(DeviceArray([ 1., -1.], dtype=float32),
 DeviceArray([4146024105,  967050713], dtype=uint32))

In [4]:
state.policy_params.keys()

dict_keys(['Normal/~/linear', 'Normal/~/linear_1', 'mlp/~/linear_0', 'mlp/~/linear_1'])

In [5]:
# the video writer commands will have to change here. I'd probably recommend using
# imageio. You just make a list of all the images, and then call imageio.mimsave(filename, image_list)

for _ in range(10):
  video_filename = '/tmp/video.mp4'
  writer = []

  vec = []
  ts = env.reset()
  env._goal_img = ts.observation.copy()
  for t in tqdm.trange(50):
    a, key = _act(ts.observation, key)
    ts = env.step(a.to_py())
    # use the following lines for image-based environments.
#     s = ts.observation[:obs_dim]
    g = ts.observation[obs_dim:]
    img = env.render(mode='rgb_array')
#     img = np.concatenate([
#       s.reshape((64, 64, 3)),
#       g.reshape((64, 64, 3))
#     ], axis=1)
    # for state-based environments, use the following
    # img = env.render(mode='rgb_array'), or something like that
    writer.append(img)
    if ts.last():
      break
  imageio.mimsave('output.gif', writer)

  mp4 = open(video_filename,'rb').read()
  data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
  html = HTML("""
  <video width=800 framerate=1 controls>
        <source src="%s" type="video/mp4">
  </video>
  """ % data_url)
  display(html)
  plt.plot(env.environment.environment.environment._dist)
  plt.show()

  0%|                                                                                                     | 0/50 [00:00<?, ?it/s]


NotImplementedError: 